In [1]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from PIL import Image
import torch
from transformers import GLPNFeatureExtractor, GLPNForDepthEstimation

feature_extractor = GLPNFeatureExtractor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

# load and resize the input image
image = Image.open("image.jpg")
new_height = 480 if image.height > 480 else image.height
new_height -= (new_height % 32)
new_width = int(new_height * image.width / image.height)
diff = new_width % 32
new_width = new_width - diff if diff < 16 else new_width + 32 - diff
new_size = (new_width, new_height)
image = image.resize(new_size)

# prepare image for the model
inputs = feature_extractor(images=image, return_tensors="pt")

# get the prediction from the model
with torch.no_grad():
    outputs = model(**inputs)
    predicted_depth = outputs.predicted_depth

# remove borders
pad = 16
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pad:-pad, pad:-pad]
image = image.crop((pad, pad, image.width - pad, image.height - pad))

# visualize the prediction
fig, ax = plt.subplots(1, 2)
ax[0].imshow(image)
ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
ax[1].imshow(output, cmap='plasma')
ax[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.tight_layout()
plt.pause(5)

/opt/homebrew/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/lib/python3.8/site-packages/transformers/models/glpn/feature_extraction_glpn.py:28: FutureWarning: The class GLPNFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use GLPNImageProcessor instead.
  warnings.warn(


In [2]:
import numpy as np
import open3d as o3d

width, height = image.size

depth_image = (output * 255 / np.max(output)).astype('uint8')
image = np.array(image)

# create rgbd image
depth_o3d = o3d.geometry.Image(depth_image)
image_o3d = o3d.geometry.Image(image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_o3d, depth_o3d, convert_rgb_to_intensity=False)

# camera settings
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsic.set_intrinsics(width, height, 500, 500, width/2, height/2)

# create point cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)

In [1]:
import trimesh
import numpy as np
from PIL import Image, ImageDraw

# Load the 3D mesh
mesh = trimesh.load('mesh.obj')

# Unfold the mesh by projecting it to 2D (UV mapping)
# This is a simplistic projection; for complex unwrapping, consider using Blender

# Use a simple orthographic projection for this example
def orthographic_projection(vertices):
    # Projecting on the XY plane
    return vertices[:, :2]

# Get the vertices and faces
vertices = mesh.vertices
faces = mesh.faces

# Project vertices to 2D
projected_vertices = orthographic_projection(vertices)

# Normalize coordinates to fit into the image dimensions
def normalize_to_image(vertices, image_size):
    min_coords = vertices.min(axis=0)
    max_coords = vertices.max(axis=0)
    normalized = (vertices - min_coords) / (max_coords - min_coords)
    return (normalized * (image_size - 1)).astype(np.int32)

image_size = 800
normalized_vertices = normalize_to_image(projected_vertices, image_size)

# Create an image and draw the mesh
image = Image.new('RGB', (image_size, image_size), 'white')
draw = ImageDraw.Draw(image)

# Draw the mesh edges
for face in faces:
    polygon = [tuple(normalized_vertices[vertex]) for vertex in face]
    draw.polygon(polygon, outline='black')

# Save and display the image
image.save('unfolded_mesh.png')
image.show()
